In [24]:
import cv2
import numpy as np
def detectLetters(img,kernelSize,rtvMode,approx):    
    boundRect = [None]
    img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    cv2.imwrite('originals/gray.png',img_gray)
    img_sobel = cv2.Sobel(img_gray,cv2.CV_8U,1,0,ksize = 1,scale=1,delta=0,borderType=cv2.BORDER_DEFAULT)
    #cv2.imwrite('originals/sobel.png',img_sobel)
    #(thresh,img_threshold) = cv2.threshold(img_sobel,0,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY)   
    
    img_threshold = cv2.threshold(img_gray,0,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)[1]
    cv2.imwrite('originals/threshold.png',img_threshold)
    #element = cv2.getStructuringElement (cv2.MORPH_RECT, (15, 3) )
    element = cv2.getStructuringElement (cv2.MORPH_RECT, kernelSize )
    img_threshold = cv2.morphologyEx(img_threshold,cv2.MORPH_CLOSE,element)
    cv2.imwrite('originals/threshold2.png',img_threshold)
    #im2,contours,hierarchy = cv2.findContours(img_threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    #im2,contours,hierarchy = cv2.findContours(img_threshold,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    im2,contours,hierarchy = cv2.findContours(img_threshold,rtvMode,approx)
    #im2,contours,hierarchy = cv2.findContours(img_threshold,0,1) 
    
    
    contours_poly = [None]*len(contours)
    for i in range(0,len(contours)):
        if len(contours[i])>100:            
            contours_poly[i]=(cv2.approxPolyDP(contours[i], 3, True))            
            x,y,w,h = cv2.boundingRect(contours_poly[i])            
            if w>h:
                temp = x,y,w,h
                boundRect.append(temp)
    return boundRect

def extractTextArea(inputFile,outputFile,kernelSize,rtvMode,approx):
    #implementation    
    img = cv2.imread(inputFile)
    print(inputFile+'=')
    print(img.shape)
    letterBBoxes=detectLetters(img,kernelSize,rtvMode,approx)
    for i in range(0,len(letterBBoxes)-1):
        if letterBBoxes[i] != None:        
            temp = letterBBoxes[i]
            cv2.rectangle(img,(temp[0],temp[1]),(temp[0]+temp[2],temp[1]+temp[3]),(0,255,0),3)
            crop_image = img[temp[1]:temp[1]+temp[3], temp[0]:temp[0]+temp[2]]
            cv2.imwrite('originals/'+str(i)+'.png',crop_image)

    cv2.imwrite(outputFile,img)
    print('all done...')


In [28]:
kernelSize = (30,30)
extractTextArea('originals/Pancard.png','originals/Pancard2.png',kernelSize,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

kernelSize = (17,3)
extractTextArea('originals/PAN-CARD.png','originals/Pancard3.png',kernelSize,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

kernelSize = (17,3)
extractTextArea('originals/Pancard_Web_2.png','originals/Pancard2_1.png',kernelSize,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

kernelSize = (17,3)
extractTextArea('originals/Pancard_Web_3.png','originals/Pancard3_1.png',kernelSize,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

originals/Pancard.png=
(1932, 3076, 3)
all done...
originals/PAN-CARD.png=
(376, 591, 3)
all done...
originals/Pancard_Web_2.png=
(835, 1146, 3)
all done...
originals/Pancard_Web_3.png=
(1424, 2178, 3)
all done...


In [3]:
import imutils
import cv2
import numpy as np
import tensorflow as tf
# load the image and compute the ratio of the old height
# to the new height, clone it, and resize it
img = cv2.imread('originals/Pancard_Web_3.png')
#increase contrast of the image
alpha = 1.7
beta = 60
X,Y,H = img.shape
image = np.empty([X,Y,H])
for x in range(0,X-1):
    for y in range(0,Y-1):
        for c in range(0,2):
            temp = (alpha*img[x,y,c])+beta
            if temp>255:
                temp = 255
            image[x,y,c] =temp
cv2.imwrite('originals/dltme.png',image)
        
print('contrast increased....')

contrast increased....


In [15]:
import imutils
import cv2
import numpy as np
import tensorflow as tf
image = cv2.imread('originals/Pancard_Web_3.png')
#image = cv2.imread('originals/Pancard.png')

ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)
 
# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)
 
# show the original image and the edge detected image
cv2.imwrite('originals/dltme1.png',edged)
print ("STEP 1: Edge Detection")

#cv2.imshow("Image", image)
#cv2.imshow("Edged", edged)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


#Binary Dialation
kernelSize = (17,3)
element = cv2.getStructuringElement (cv2.MORPH_RECT, kernelSize )
edged = cv2.morphologyEx(edged,cv2.MORPH_CLOSE,element)
cv2.imwrite('originals/dltme2.png',edged)
print('Binary dialation done...')

# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
#(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
(_,cnts,_ ) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
 
# loop over the contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break
 
# show the contour (outline) of the piece of paper
print ("STEP 2: Find contours of paper")

cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imwrite('originals/dltme3.png',image)
print('Contours done')
#cv2.imshow("Outline", image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

STEP 1: Edge Detection
Binary dialation done...
STEP 2: Find contours of paper
Contours done


STEP 2: Find contours of paper
